In [8]:
#Alle Programme instalieren / importieren
import pandas as pd
import numpy as np

from keplergl import KeplerGl 
import keplergl

#Bokeh wir nicht benötigt

#fiona in anaconda Installieren
#Grundlage: https://medium.com/vis-gl/introducing-kepler-gl-for-jupyter-f72d41659fbf
#Einlesen der Daten 
file_path = r"C:\Users\franz\OneDrive\Desktop\TUHH\3. Semester\00_TechLabs\data\Sauber/Unfallorte_gesamt_bereinigt.txt"


In [9]:
#Dateneinlesen in ein DateFrame
df = pd.read_csv(file_path, sep=",")




In [10]:
#Anaconda auf Updates überprüfen
#!conda update conda
#Instalaltionen
#!pip install keplergl
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager keplergl-jupyter

In [11]:
#Daten überprüfen
df.describe()




,Unnamed: 0,OBJECTID,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,IstKrad,IstGkfz,IstSonstig,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,UIDENTSTLA,lat_short,long_short
count,9019.000000,9019.000000,9019.0,9019.000000,9019.000000,9019.000000,9019.000000,9019.000000,9019.000000,9019.000000,...,9019.000000,6839.000000,9019.000000,9019.000000,9.019000e+03,9019.000000,9019.000000,2.180000e+03,9019.000000,9019.000000
mean,114267.746757,114268.746757,2.0,3.336179,6.573013,348.808848,2017.525225,6.855305,13.233285,4.062535,...,0.010201,0.020471,0.088480,566613.368278,5.936381e+06,10.005960,53.572028,2.170692e+18,10.005963,53.572026
std,42798.115215,42798.115215,0.0,1.702043,4.626534,168.811413,1.118456,2.949343,4.578789,1.696236,...,0.100488,0.141615,0.284007,5714.901154,5.022940e+03,0.086317,0.045124,2.962057e+14,0.086325,0.045124
min,6022.000000,6023.000000,2.0,1.000000,1.000000,101.000000,2016.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,549085.167200,5.917209e+06,9.741474,53.397676,2.170102e+18,9.741000,53.398000
25%,97696.000000,97697.000000,2.0,2.000000,3.000000,212.000000,2017.000000,5.000000,9.000000,3.000000,...,0.000000,0.000000,0.000000,563113.830901,5.934276e+06,9.953118,53.553348,2.170503e+18,9.953000,53.553000
50%,124490.000000,124491.000000,2.0,3.000000,5.000000,320.000000,2018.000000,7.000000,14.000000,4.000000,...,0.000000,0.000000,0.000000,565984.564800,5.936364e+06,9.996110,53.572078,2.170706e+18,9.996000,53.572000
75%,128306.000000,128307.000000,2.0,5.000000,9.000000,508.000000,2019.000000,9.000000,17.000000,5.000000,...,0.000000,0.000000,0.000000,569968.091350,5.939114e+06,10.056940,53.596609,2.170920e+18,10.057000,53.597000
max,163330.000000,163331.000000,2.0,7.000000,20.000000,718.000000,2019.000000,12.000000,23.000000,7.000000,...,1.000000,1.000000,1.000000,584611.109122,5.952865e+06,10.273421,53.719033,2.171231e+18,10.273000,53.719000


In [12]:
#Configuration der Karte einrichten (Startposition)
config = {
'version': 'v1',
'config': {
    'mapState': {
        'latitude': 53.6,
        'longitude': 10,
        'zoom': 8
    }}}


In [13]:
#Karten inklusive der Daten
map_1 = KeplerGl(height=800, data={"data_1": df},config=config)


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [14]:
#aufrufen der Karte ES muss ein neuer Layer eingestellt werden
map_1


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 53.6, 'longitude': 10, 'zoom': 8}}}, dat…

In [17]:
{
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "xiis2o5",
          "type": "heatmap",
          "config": {
            "dataId": "data_1",
            "label": "new layer",
            "color": [
              255,
              203,
              153
            ],
            "columns": {
              "lat": "YGCSWGS84",
              "lng": "XGCSWGS84"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.8,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 5.1
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "weightField": None,
            "weightScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "data_1": [
              {
                "name": "Unnamed: 0",
                "format": None
              },
              {
                "name": "OBJECTID",
                "format": None
              },
              {
                "name": "ULAND",
                "format": None
              },
              {
                "name": "UREGBEZ",
                "format": None
              },
              {
                "name": "UKREIS",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 53.43430162176991,
      "longitude": 10.2508837414174,
      "pitch": 0,
      "zoom": 7.522302737901885,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'xiis2o5',
     'type': 'heatmap',
     'config': {'dataId': 'data_1',
      'label': 'new layer',
      'color': [255, 203, 153],
      'columns': {'lat': 'YGCSWGS84', 'lng': 'XGCSWGS84'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 5.1},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'weightField': None, 'weightScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'data_1': [{'name': 'Unnamed: 0',
        'format': None},
       {'name': 'OBJECT